# 分類任務 (Classification)

## 學習目標
- 理解分類問題與迴歸問題的差異
- 掌握多類別分類的實現
- 學習音框分類任務（李宏毅 HW2）
- 處理類別不平衡問題

## 對應課程
- [李宏毅 ML 2021 - Lecture 2: Classification](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW2: Phoneme Classification](https://github.com/ga642381/ML2021-Spring/tree/main/HW02)

## 分類問題概述

```
分類任務類型
├── 二元分類 (Binary): 垃圾郵件檢測
├── 多類別分類 (Multi-class): 手寫數字識別（0-9）
└── 多標籤分類 (Multi-label): 文章標籤（可有多個標籤）

HW2: Phoneme Classification
├── 輸入: 語音特徵 (MFCC)
├── 輸出: 39 個音素類別之一
└── 挑戰: 上下文相關性、類別不平衡
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

---
## Part 1: 分類基礎

### 分類 vs 迴歸

| | 迴歸 | 分類 |
|---|---|---|
| **輸出** | 連續值 | 離散類別 |
| **損失函數** | MSE, MAE | Cross-Entropy |
| **輸出層** | 1 個神經元 | N 個神經元 (N 類) |
| **激活函數** | 無 / Linear | Softmax |

In [ ]:
# ========== Softmax 與 Cross-Entropy ==========

# Softmax: 將 logits 轉為概率分布
logits = torch.tensor([2.0, 1.0, 0.1])
probs = F.softmax(logits, dim=0)

print("Softmax 示例:")
print(f"  Logits: {logits}")
print(f"  Probabilities: {probs}")
print(f"  Sum: {probs.sum():.4f}")

# Cross-Entropy Loss
# PyTorch 的 CrossEntropyLoss 包含 Softmax，輸入應為 logits
criterion = nn.CrossEntropyLoss()

# 模擬批次資料
batch_logits = torch.randn(4, 3)  # 4 個樣本，3 個類別
targets = torch.tensor([0, 1, 2, 0])  # 真實標籤

loss = criterion(batch_logits, targets)
print(f"\nCross-Entropy Loss: {loss.item():.4f}")

In [ ]:
# ========== 分類模型基本結構 ==========

class Classifier(nn.Module):
    """
    基本分類器
    
    注意：輸出層不加 Softmax，因為 CrossEntropyLoss 會處理
    """
    
    def __init__(self, input_dim, hidden_dims, num_classes):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.2),
            ])
            prev_dim = hidden_dim
        
        # 輸出層：num_classes 個類別
        layers.append(nn.Linear(prev_dim, num_classes))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)  # 輸出 logits

# 測試
model = Classifier(input_dim=39, hidden_dims=[256, 128, 64], num_classes=39)
print(model)

x = torch.randn(4, 39)
out = model(x)
print(f"\n輸入: {x.shape}")
print(f"輸出 (logits): {out.shape}")
print(f"預測類別: {out.argmax(dim=1)}")

---
## Part 2: Phoneme Classification（李宏毅 HW2）

### 任務說明

**目標**：根據語音特徵 (MFCC) 預測音素類別

**特徵**：
- MFCC (Mel-Frequency Cepstral Coefficients): 39 維
- 可考慮前後幾個 frame 的上下文

**輸出**：39 個音素類別之一

In [ ]:
# ========== 模擬 Phoneme 資料集 ==========

def generate_phoneme_data(n_samples=10000, feat_dim=39, n_classes=39, context=5):
    """
    生成類似 HW2 的 Phoneme 資料
    
    實際 HW2:
    - 每個 frame 有 39 維 MFCC 特徵
    - 可以串接前後 n 個 frame 作為上下文
    - 目標是 39 類音素
    """
    np.random.seed(42)
    
    # 特徵維度 = MFCC * (context * 2 + 1)
    total_feat_dim = feat_dim * (context * 2 + 1)
    
    # 生成特徵（模擬 MFCC）
    X = np.random.randn(n_samples, total_feat_dim).astype(np.float32)
    
    # 生成標籤（模擬不平衡分布）
    # 某些音素出現頻率較高
    class_probs = np.random.dirichlet(np.ones(n_classes) * 0.5)
    y = np.random.choice(n_classes, n_samples, p=class_probs).astype(np.int64)
    
    return X, y

# 生成資料
CONTEXT = 5  # 考慮前後各 5 個 frame
X, y = generate_phoneme_data(n_samples=10000, context=CONTEXT)

print(f"特徵維度: {X.shape}")
print(f"  MFCC: 39 維")
print(f"  上下文: 前後各 {CONTEXT} 個 frame")
print(f"  總維度: 39 × {CONTEXT*2+1} = {39*(CONTEXT*2+1)}")
print(f"\n標籤分布:")
label_counts = Counter(y)
print(f"  類別數: {len(label_counts)}")
print(f"  最多: {max(label_counts.values())}")
print(f"  最少: {min(label_counts.values())}")

In [ ]:
# ========== Dataset 類別 ==========

class PhonemeDataset(Dataset):
    """
    Phoneme 分類資料集
    """
    
    def __init__(self, X, y=None):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y) if y is not None else None
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# 分割資料
train_ratio = 0.8
n_train = int(len(X) * train_ratio)

# 標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X[:n_train])
X_val = scaler.transform(X[n_train:])
y_train, y_val = y[:n_train], y[n_train:]

print(f"訓練集: {X_train.shape[0]}")
print(f"驗證集: {X_val.shape[0]}")

In [ ]:
# ========== 進階分類器（HW2 Strong Baseline）==========

class PhonemeClassifier(nn.Module):
    """
    Phoneme 分類器
    
    HW2 Strong Baseline 技巧:
    - 較深的網路 (5-6 層)
    - BatchNorm + Dropout
    - 適當的 hidden size
    """
    
    def __init__(self, input_dim, hidden_dim=512, num_classes=39, num_layers=5, dropout=0.25):
        super().__init__()
        
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        
        # 中間層
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_layers - 2):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ))
        
        # 輸出層
        self.output_layer = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = self.input_layer(x)
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)

# 建立模型
model = PhonemeClassifier(
    input_dim=X_train.shape[1],
    hidden_dim=512,
    num_classes=39,
    num_layers=5,
    dropout=0.25
)

# 計算參數量
total_params = sum(p.numel() for p in model.parameters())
print(f"模型參數量: {total_params:,}")

In [ ]:
# ========== 訓練配置 ==========

config = {
    'batch_size': 512,
    'epochs': 50,
    'lr': 1e-3,
    'weight_decay': 1e-5,
}

# 建立 DataLoader
train_dataset = PhonemeDataset(X_train, y_train)
val_dataset = PhonemeDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

In [ ]:
# ========== 訓練函數 ==========

def train_classifier(model, train_loader, val_loader, config):
    """
    訓練分類模型
    """
    model = model.to(device)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=config['lr'],
        weight_decay=config['weight_decay']
    )
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config['epochs']
    )
    
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0
    
    for epoch in range(config['epochs']):
        # ========== 訓練 ==========
        model.train()
        train_loss = 0
        
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * len(X_batch)
        
        train_loss /= len(train_loader.dataset)
        
        # ========== 驗證 ==========
        model.eval()
        val_loss = 0
        correct = 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                
                logits = model(X_batch)
                loss = criterion(logits, y_batch)
                
                val_loss += loss.item() * len(X_batch)
                pred = logits.argmax(dim=1)
                correct += (pred == y_batch).sum().item()
        
        val_loss /= len(val_loader.dataset)
        val_acc = correct / len(val_loader.dataset)
        
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_state = model.state_dict().copy()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{config['epochs']}")
            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    model.load_state_dict(best_model_state)
    print(f"\n最佳驗證準確率: {best_acc:.4f}")
    
    return model, history

# 訓練
model, history = train_classifier(model, train_loader, val_loader, config)

In [ ]:
# ========== 結果可視化 ==========

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 損失曲線
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 準確率曲線
axes[1].plot(history['val_acc'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy')
axes[1].grid(True, alpha=0.3)

# 混淆矩陣
model.eval()
with torch.no_grad():
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_pred = model(X_val_tensor).argmax(dim=1).cpu().numpy()

# 只顯示部分類別的混淆矩陣
cm = confusion_matrix(y_val[:1000], y_pred[:1000])
sns.heatmap(cm[:10, :10], annot=True, fmt='d', cmap='Blues', ax=axes[2])
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('Actual')
axes[2].set_title('Confusion Matrix (Top 10 Classes)')

plt.tight_layout()
plt.show()

# 分類報告
print("\n分類報告 (前 10 類):")
print(classification_report(y_val, y_pred, labels=range(10), digits=3))

---
## Part 3: 處理類別不平衡

實際資料中，各類別樣本數往往不平衡。

In [ ]:
# ========== 方法 1: Weighted Loss ==========

def compute_class_weights(labels, num_classes):
    """
    計算類別權重
    
    少數類別獲得更高權重
    """
    counts = np.bincount(labels, minlength=num_classes)
    weights = 1.0 / (counts + 1e-6)
    weights = weights / weights.sum() * num_classes
    return torch.FloatTensor(weights)

# 計算權重
class_weights = compute_class_weights(y_train, 39)
print(f"類別權重範圍: [{class_weights.min():.2f}, {class_weights.max():.2f}]")

# 使用加權損失
weighted_criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [ ]:
# ========== 方法 2: Weighted Sampler ==========

def get_weighted_sampler(labels):
    """
    建立加權採樣器
    
    使少數類別被採樣的機率增加
    """
    counts = np.bincount(labels)
    weights = 1.0 / counts[labels]
    weights = torch.DoubleTensor(weights)
    
    sampler = WeightedRandomSampler(
        weights=weights,
        num_samples=len(weights),
        replacement=True
    )
    return sampler

# 使用加權採樣器
sampler = get_weighted_sampler(y_train)
balanced_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    sampler=sampler  # 不能同時使用 shuffle 和 sampler
)

# 驗證採樣效果
batch_labels = []
for _, y_batch in balanced_loader:
    batch_labels.extend(y_batch.tolist())
    if len(batch_labels) > 2000:
        break

print("加權採樣後的類別分布更均勻")
print(f"原始最大/最小比: {max(Counter(y_train).values()) / min(Counter(y_train).values()):.2f}")
print(f"採樣後最大/最小比: {max(Counter(batch_labels).values()) / min(Counter(batch_labels).values()):.2f}")

In [ ]:
# ========== 方法 3: Focal Loss ==========

class FocalLoss(nn.Module):
    """
    Focal Loss: 降低簡單樣本的權重，專注於困難樣本
    
    FL(p_t) = -α_t * (1 - p_t)^γ * log(p_t)
    
    - γ (gamma): 聚焦參數，γ=0 時等同於 CE Loss
    - α (alpha): 類別平衡因子
    """
    
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_weight = (1 - p_t) ** self.gamma
        
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_weight = alpha_t * focal_weight
        
        loss = focal_weight * ce_loss
        
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

# 測試 Focal Loss
focal_criterion = FocalLoss(gamma=2.0)
ce_criterion = nn.CrossEntropyLoss()

# 比較兩種損失
logits = torch.randn(100, 39)
targets = torch.randint(0, 39, (100,))

print(f"Cross-Entropy Loss: {ce_criterion(logits, targets):.4f}")
print(f"Focal Loss: {focal_criterion(logits, targets):.4f}")

---
## Part 4: Label Smoothing

防止模型過度自信。

In [ ]:
# ========== Label Smoothing ==========

class LabelSmoothingCrossEntropy(nn.Module):
    """
    Label Smoothing Cross Entropy
    
    將 one-hot 標籤軟化：
    - 原本: [0, 0, 1, 0, 0]
    - 軟化: [0.02, 0.02, 0.92, 0.02, 0.02] (smoothing=0.1)
    """
    
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    
    def forward(self, inputs, targets):
        n_classes = inputs.size(-1)
        log_preds = F.log_softmax(inputs, dim=-1)
        
        # 計算 smooth label 的損失
        loss = -log_preds.sum(dim=-1) / n_classes
        
        # 計算 hard label 的損失
        nll_loss = F.nll_loss(log_preds, targets, reduction='none')
        
        # 組合
        loss = (1 - self.smoothing) * nll_loss + self.smoothing * loss
        return loss.mean()

# PyTorch 內建版本（1.10+）
smooth_criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

print("Label Smoothing 效果:")
print(f"  CE Loss: {ce_criterion(logits, targets):.4f}")
print(f"  Smooth CE Loss: {smooth_criterion(logits, targets):.4f}")

---
## 🎯 總結

### 分類任務重點

| 技巧 | 說明 |
|------|------|
| **Cross-Entropy Loss** | 分類標準損失函數 |
| **Softmax** | 輸出轉為概率（PyTorch CE 內建）|
| **BatchNorm + Dropout** | 正則化 |
| **Weighted Loss** | 處理類別不平衡 |
| **Label Smoothing** | 防止過度自信 |

### 評估指標

| 指標 | 說明 |
|------|------|
| **Accuracy** | 整體準確率 |
| **Precision** | 預測為正的正確比例 |
| **Recall** | 實際為正的檢出比例 |
| **F1-Score** | Precision 和 Recall 的調和平均 |
| **Confusion Matrix** | 詳細類別預測情況 |

### 李宏毅 HW2 技巧

```
Strong Baseline 達成方法:
1. 串接上下文 frame（前後各 5-11 個）
2. 較深的網路（5-6 層）
3. 較大的 hidden size（512-1024）
4. BatchNorm + Dropout
5. 學習率調度（Cosine Annealing）
```

### 下一步

前往 `computer_vision/cnn_fundamentals.ipynb` 學習卷積神經網路！

---
## 練習題

### 練習 1: 比較不同上下文長度的效果

**目標**: 實驗不同 context 大小對音框分類的影響

In [ ]:
# 練習 1: 比較不同上下文長度

def experiment_context_size(context_sizes=[0, 2, 5, 8, 11]):
    """比較不同上下文長度的效果"""
    results = []
    
    for ctx in context_sizes:
        print(f"\n=== Context = {ctx} ===")
        
        # 生成資料
        X_ctx, y_ctx = generate_phoneme_data(n_samples=5000, context=ctx)
        
        # 分割
        n_train = int(len(X_ctx) * 0.8)
        X_tr, X_vl = X_ctx[:n_train], X_ctx[n_train:]
        y_tr, y_vl = y_ctx[:n_train], y_ctx[n_train:]
        
        # 標準化
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr)
        X_vl = scaler.transform(X_vl)
        
        # 建立 DataLoader
        train_ds = PhonemeDataset(X_tr, y_tr)
        val_ds = PhonemeDataset(X_vl, y_vl)
        train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
        val_dl = DataLoader(val_ds, batch_size=256)
        
        # 訓練模型（簡化版）
        model = PhonemeClassifier(input_dim=X_tr.shape[1], hidden_dim=256, num_layers=3).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(20):
            model.train()
            for X_batch, y_batch in train_dl:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()
        
        # 評估
        model.eval()
        correct = 0
        with torch.no_grad():
            for X_batch, y_batch in val_dl:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                pred = model(X_batch).argmax(dim=1)
                correct += (pred == y_batch).sum().item()
        
        acc = correct / len(val_ds)
        results.append({'context': ctx, 'accuracy': acc, 'feature_dim': X_tr.shape[1]})
        print(f"  Feature dim: {X_tr.shape[1]}, Accuracy: {acc:.4f}")
    
    return results

# 執行實驗
ctx_results = experiment_context_size()

# 視覺化
fig, ax = plt.subplots(figsize=(8, 5))
contexts = [r['context'] for r in ctx_results]
accs = [r['accuracy'] for r in ctx_results]
ax.plot(contexts, accs, 'bo-', linewidth=2, markersize=10)
ax.set_xlabel('Context Size')
ax.set_ylabel('Validation Accuracy')
ax.set_title('Effect of Context Size on Phoneme Classification')
ax.grid(True, alpha=0.3)
plt.show()

print(f"\n最佳 context: {ctx_results[np.argmax(accs)]['context']}")

### 練習 2: 比較不同處理類別不平衡的方法

**目標**: 比較 Weighted Loss、Focal Loss、Weighted Sampler 的效果

In [ ]:
# 練習 2: 比較處理類別不平衡的方法

# 生成高度不平衡的資料
np.random.seed(42)
n_samples = 3000
n_classes = 10

# 極度不平衡：類別 0 有 50% 樣本，其他均分
class_probs = np.array([0.5] + [0.5 / (n_classes - 1)] * (n_classes - 1))
X_imb = np.random.randn(n_samples, 20).astype(np.float32)
y_imb = np.random.choice(n_classes, n_samples, p=class_probs).astype(np.int64)

print("類別分布:")
for i, cnt in enumerate(np.bincount(y_imb)):
    print(f"  Class {i}: {cnt} ({cnt/n_samples*100:.1f}%)")

# 分割
n_tr = int(n_samples * 0.8)
X_tr, X_vl = X_imb[:n_tr], X_imb[n_tr:]
y_tr, y_vl = y_imb[:n_tr], y_imb[n_tr:]

def train_with_method(method_name, train_loader, criterion, epochs=30):
    """使用指定方法訓練"""
    model = nn.Sequential(
        nn.Linear(20, 64), nn.ReLU(), nn.Dropout(0.2),
        nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.2),
        nn.Linear(32, n_classes)
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for _ in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            loss = criterion(model(X_batch), y_batch)
            loss.backward()
            optimizer.step()
    
    # 評估
    model.eval()
    with torch.no_grad():
        X_vl_t = torch.FloatTensor(X_vl).to(device)
        pred = model(X_vl_t).argmax(dim=1).cpu().numpy()
    
    # 計算各類別的準確率
    acc_per_class = []
    for c in range(n_classes):
        mask = y_vl == c
        if mask.sum() > 0:
            acc_per_class.append((pred[mask] == c).mean())
        else:
            acc_per_class.append(0)
    
    return {
        'overall_acc': (pred == y_vl).mean(),
        'macro_acc': np.mean(acc_per_class),
        'per_class': acc_per_class
    }

# 準備資料集
train_ds = PhonemeDataset(X_tr, y_tr)
val_ds = PhonemeDataset(X_vl, y_vl)

# 方法 1: 基準（無處理）
baseline_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
baseline_result = train_with_method("Baseline", baseline_loader, nn.CrossEntropyLoss())

# 方法 2: Weighted Loss
class_weights = compute_class_weights(y_tr, n_classes).to(device)
weighted_result = train_with_method("Weighted Loss", baseline_loader, 
                                     nn.CrossEntropyLoss(weight=class_weights))

# 方法 3: Focal Loss
focal_result = train_with_method("Focal Loss", baseline_loader, FocalLoss(gamma=2.0))

# 方法 4: Weighted Sampler
sampler = get_weighted_sampler(y_tr)
sampler_loader = DataLoader(train_ds, batch_size=64, sampler=sampler)
sampler_result = train_with_method("Weighted Sampler", sampler_loader, nn.CrossEntropyLoss())

# 比較結果
print("\n=== 結果比較 ===")
print(f"{'方法':<20} {'Overall Acc':<12} {'Macro Acc':<12}")
print("-" * 44)
for name, res in [("Baseline", baseline_result), ("Weighted Loss", weighted_result),
                   ("Focal Loss", focal_result), ("Weighted Sampler", sampler_result)]:
    print(f"{name:<20} {res['overall_acc']:.4f}       {res['macro_acc']:.4f}")

# 視覺化各類別準確率
fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(n_classes)
width = 0.2

ax.bar(x - 1.5*width, baseline_result['per_class'], width, label='Baseline')
ax.bar(x - 0.5*width, weighted_result['per_class'], width, label='Weighted Loss')
ax.bar(x + 0.5*width, focal_result['per_class'], width, label='Focal Loss')
ax.bar(x + 1.5*width, sampler_result['per_class'], width, label='Weighted Sampler')

ax.set_xlabel('Class')
ax.set_ylabel('Accuracy')
ax.set_title('Per-Class Accuracy Comparison')
ax.set_xticks(x)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.show()

print("\n結論: 處理類別不平衡可以顯著提升少數類別的準確率")